In [ ]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import load_dataset

data = load_dataset('ncbi_disease')
data

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5433 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/924 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/941 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 5433
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 924
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 941
    })
})

In [ ]:
print('ID',data['train'][:]['id'][0])
print('tokens',data['train'][:]['tokens'][0])
print('ner_tags',data['train'][:]['ner_tags'][0])

ID 0
tokens ['Identification', 'of', 'APC2', ',', 'a', 'homologue', 'of', 'the', 'adenomatous', 'polyposis', 'coli', 'tumour', 'suppressor', '.']
ner_tags [0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0]


In [ ]:
tags = data['train'].features['ner_tags'].feature

print(tags)

ClassLabel(names=['O', 'B-Disease', 'I-Disease'], id=None)


In [ ]:
index2tag = {idx:tag for idx, tag in enumerate(tags.names)}
print(index2tag)

{0: 'O', 1: 'B-Disease', 2: 'I-Disease'}


In [ ]:
tag2index = {tag:idx for idx, tag in enumerate(tags.names)}
print(tag2index)

{'O': 0, 'B-Disease': 1, 'I-Disease': 2}


In [ ]:
tags.int2str(2)

'I-Disease'

In [ ]:
def create_tag_names(batch):
  tag_name = {'ner_tags_str': [tags.int2str(idx) for idx in batch['ner_tags']]}
  return tag_name

In [ ]:
data = data.map(create_tag_names)

Map:   0%|          | 0/5433 [00:00<?, ? examples/s]

Map:   0%|          | 0/924 [00:00<?, ? examples/s]

Map:   0%|          | 0/941 [00:00<?, ? examples/s]

In [ ]:
data['train'][:].keys()

dict_keys(['id', 'tokens', 'ner_tags', 'ner_tags_str'])

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "dmis-lab/biobert-base-cased-v1.2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [ ]:
inputs = data['train'][0]['tokens']
inputs

['Identification',
 'of',
 'APC2',
 ',',
 'a',
 'homologue',
 'of',
 'the',
 'adenomatous',
 'polyposis',
 'coli',
 'tumour',
 'suppressor',
 '.']

In [ ]:
inputs = tokenizer(inputs, is_split_into_words=True)
inputs

{'input_ids': [101, 9117, 1104, 170, 1643, 1665, 1477, 117, 170, 16358, 3702, 12733, 1104, 1103, 8050, 26601, 21943, 2285, 185, 23415, 5674, 4863, 1884, 2646, 189, 27226, 17203, 1766, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
inputs.tokens()

['[CLS]',
 'identification',
 'of',
 'a',
 '##p',
 '##c',
 '##2',
 ',',
 'a',
 'ho',
 '##mo',
 '##logue',
 'of',
 'the',
 'ad',
 '##eno',
 '##mat',
 '##ous',
 'p',
 '##oly',
 '##po',
 '##sis',
 'co',
 '##li',
 't',
 '##umour',
 'suppress',
 '##or',
 '.',
 '[SEP]']

In [ ]:
def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word=None
  for word_id in word_ids:
    if word_id != current_word:
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)

    elif word_id is None:
      new_labels.append(-100)

    else:
      label = labels[word_id]

      if label%2==1:
        label = label + 1
      new_labels.append(label)

  return new_labels

In [ ]:
labels = data['train'][0]['ner_tags']
word_ids = inputs.word_ids()
print(labels, word_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0] [None, 0, 1, 2, 2, 2, 2, 3, 4, 5, 5, 5, 6, 7, 8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 11, 11, 12, 12, 13, None]


In [ ]:
align_labels_with_tokens(labels, word_ids)

[-100,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 -100]

In [ ]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

  all_labels = examples['ner_tags']

  new_labels = []
  for i, labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels, word_ids))

  tokenized_inputs['labels'] = new_labels

  return tokenized_inputs

In [ ]:
tokenized_datasets = data.map(tokenize_and_align_labels, batched=True, remove_columns=data['train'].column_names)

Map:   0%|          | 0/5433 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/924 [00:00<?, ? examples/s]

Map:   0%|          | 0/941 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5433
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 924
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 941
    })
})

In [ ]:
print('input_ids',tokenized_datasets['train'][:]['input_ids'][0])
print('attention_mask',tokenized_datasets['train'][:]['attention_mask'][0])
print('labels',tokenized_datasets['train'][:]['labels'][0])

print('length:')
print(len(tokenized_datasets['train'][:]['input_ids'][0]))
print(len(tokenized_datasets['train'][:]['attention_mask'][0]))
print(len(tokenized_datasets['train'][:]['labels'][0]))

input_ids [101, 9117, 1104, 170, 1643, 1665, 1477, 117, 170, 16358, 3702, 12733, 1104, 1103, 8050, 26601, 21943, 2285, 185, 23415, 5674, 4863, 1884, 2646, 189, 27226, 17203, 1766, 119, 102]
attention_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
labels [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, -100]
length:
30
30
30


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

batch = data_collator([tokenized_datasets['train'][i] for i in range(2)])
batch

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[  101,  9117,  1104,   170,  1643,  1665,  1477,   117,   170, 16358,
          3702, 12733,  1104,  1103,  8050, 26601, 21943,  2285,   185, 23415,
          5674,  4863,  1884,  2646,   189, 27226, 17203,  1766,   119,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
        [  101,  1103,  8050, 26601, 21943,  2285,   185, 23415,  5674,  4863,
          1884,  2646,   113,   170,  1643,  1665,   114,   189, 27226,   118,
         17203,  1766,  4592,  7451,  1103,   192,  2227, 25498, 13548,  1118,
          5071,   170,  2703,  1114,   176,  1193,  2528,  4915,   188, 26588,
          6530, 24779,   124, 16632,  1161,   113,   176,  5276,   118,   124,
         16632,  1161,   114,   117,   170, 16594,   120,  5880,  1394, 

In [ ]:
!pip install seqeval
!pip install evaluate

import evaluate
metric = evaluate.load('seqeval')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 904.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=e1dba535ddbfde60006d5e72ef77010c11c3784ccf9cfbf2cfa4c568ac9906ea
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00


In [ ]:
ner_feature = data['train'].features['ner_tags']
ner_feature

label_names = ner_feature.feature.names
label_names


['O', 'B-Disease', 'I-Disease']

In [ ]:
labels = data['train'][0]['ner_tags']
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Disease',
 'I-Disease',
 'I-Disease',
 'I-Disease',
 'O',
 'O']

In [ ]:
predictions = labels.copy()
predictions[2] = "O"

metric.compute(predictions=[predictions], references=[labels])

{'Disease': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
import numpy as np

def compute_metrics(eval_preds):
  logits, labels = eval_preds

  predictions = np.argmax(logits, axis=-1)

  true_labels = [[label_names[l] for l in label if l!=-100] for label in labels]

  true_predictions = [[label_names[p] for p,l in zip(prediction, label) if l!=-100]
                      for prediction, label in zip(predictions, labels)]

  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

  return {"precision": all_metrics['overall_precision'],
          "recall": all_metrics['overall_recall'],
          "f1": all_metrics['overall_f1'],
          "accuracy": all_metrics['overall_accuracy']}

In [ ]:
id2label = {i:label for i, label in enumerate(label_names)}
label2id = {label:i for i, label in enumerate(label_names)}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
                                                    model_checkpoint,
                                                    id2label=id2label,
                                                    label2id=label2id)

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.num_labels

3

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments("dmis-lab/biobert-base-cased-v1.2",
                         evaluation_strategy = "epoch",
                         save_strategy="epoch",
                         learning_rate = 2e-5,
                         num_train_epochs=15,
                         weight_decay=0.01)

In [ ]:
from transformers import Trainer
trainer = Trainer(model=model,
                  args=args,
                  train_dataset = tokenized_datasets['train'],
                  eval_dataset = tokenized_datasets['validation'],
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.103200,0.053159,0.755556,0.864041,0.806165,0.981954
2,0.042100,0.064052,0.741031,0.839898,0.787373,0.981273
3,0.016400,0.061385,0.817284,0.841169,0.829054,0.982976
4,0.011300,0.068402,0.816133,0.874206,0.844172,0.982821
5,0.006700,0.095234,0.810157,0.851334,0.830235,0.981831
6,0.003300,0.104803,0.832918,0.848793,0.840780,0.983719
7,0.002500,0.107699,0.839752,0.858958,0.849246,0.984028
8,0.002300,0.103191,0.834751,0.872935,0.853416,0.984585
9,0.000500,0.114101,0.833534,0.878018,0.855198,0.984462
10,0.001100,0.125355,0.843945,0.858958,0.851385,0.984152


TrainOutput(global_step=10200, training_loss=0.011166917904298388, metrics={'train_runtime': 1450.4432, 'train_samples_per_second': 56.186, 'train_steps_per_second': 7.032, 'total_flos': 2668474384462872.0, 'train_loss': 0.011166917904298388, 'epoch': 15.0})

In [ ]:
from transformers import pipeline

checkpoint = "/content/dmis-lab/biobert-base-cased-v1.2/checkpoint-680"
token_classifier = pipeline(
    "token-classification", model=checkpoint, aggregation_strategy="simple"
)
print("Sentence:",' '.join(data['train'][0]['tokens']))
print('Actual labels',data['train'][0]['ner_tags_str'])
token_classifier(' '.join(data['train'][0]['tokens']))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentence: Identification of APC2 , a homologue of the adenomatous polyposis coli tumour suppressor .
Actual labels ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Disease', 'I-Disease', 'I-Disease', 'I-Disease', 'O', 'O']


[{'entity_group': 'Disease',
  'score': 0.9915742,
  'word': 'adenomatous polyposis coli tumour',
  'start': 44,
  'end': 77}]

In [ ]:
sentence = 'Several MEDIC entries may match the mention “bone dysplasia”: Bone Diseases, Developmental [MESH:D001848], Gracile bone dysplasia [MESH:C537291], Fibrous Dysplasia of Bone [MESH:D005357], etc. For the mention “bone dysplasia”, the entry MESH:D001848 is selected because it lists a synonym which is a perfect match to this mention.'
print("Sentence:",sentence)
token_classifier(sentence)

Sentence: Several MEDIC entries may match the mention “bone dysplasia”: Bone Diseases, Developmental [MESH:D001848], Gracile bone dysplasia [MESH:C537291], Fibrous Dysplasia of Bone [MESH:D005357], etc. For the mention “bone dysplasia”, the entry MESH:D001848 is selected because it lists a synonym which is a perfect match to this mention.


[{'entity_group': 'Disease',
  'score': 0.99054015,
  'word': 'bone dysplasia',
  'start': 45,
  'end': 59},
 {'entity_group': 'Disease',
  'score': 0.9567399,
  'word': 'bone diseases',
  'start': 62,
  'end': 75},
 {'entity_group': 'Disease',
  'score': 0.9829403,
  'word': 'gracile bone dysplasia',
  'start': 107,
  'end': 129},
 {'entity_group': 'Disease',
  'score': 0.9940803,
  'word': 'fibrous dysplasia of bone',
  'start': 146,
  'end': 171},
 {'entity_group': 'Disease',
  'score': 0.9912958,
  'word': 'bone dysplasia',
  'start': 210,
  'end': 224}]

In [ ]:
sentence = 'The NLRP3 inflammasome is an intracellular sensor and an essential component of the innate immune system involved in danger recognition. An important hallmark of inflammasome activation is the formation of a single supramolecular punctum, known as a speck, per cell, which is the site where the pro-inflammatory cytokines IL-1β and IL-18 are converted into their bioactive form. Speck also provides the platform for gasdermin D protein activation, whose N-terminus domain perforates the plasma membrane, allowing the release of mature cytokines alongside with a highly inflammatory form of cell death, namely pyroptosis. Although controlled NLRP3 inflammasome-pyroptosis pathway activation preserves mucosal immunity homeostasis and contributes to host defense, a prolonged trigger is deleterious and could lead, in genetically predisposed subjects, to the onset of inflammatory bowel disease, including Crohns disease and ulcerative colitis, as well as to gastrointestinal cancer. Experimental evidence shows that the NLRP3 inflammasome has both protective and pathogenic abilities. In this review we highlight the impact of the NLRP3-pyroptosis axis on the pathophysiology of the gastrointestinal tract at molecular level, focusing on newly discovered features bearing pro- and anti-inflammatory and neoplastic activity, and on targeted therapies tested in preclinical and clinical trials.'
print("Sentence:",sentence)
token_classifier(sentence)

Sentence: The NLRP3 inflammasome is an intracellular sensor and an essential component of the innate immune system involved in danger recognition. An important hallmark of inflammasome activation is the formation of a single supramolecular punctum, known as a speck, per cell, which is the site where the pro-inflammatory cytokines IL-1β and IL-18 are converted into their bioactive form. Speck also provides the platform for gasdermin D protein activation, whose N-terminus domain perforates the plasma membrane, allowing the release of mature cytokines alongside with a highly inflammatory form of cell death, namely pyroptosis. Although controlled NLRP3 inflammasome-pyroptosis pathway activation preserves mucosal immunity homeostasis and contributes to host defense, a prolonged trigger is deleterious and could lead, in genetically predisposed subjects, to the onset of inflammatory bowel disease, including Crohns disease and ulcerative colitis, as well as to gastrointestinal cancer. Experime

[{'entity_group': 'Disease',
  'score': 0.82536197,
  'word': 'pyroptosis',
  'start': 609,
  'end': 619},
 {'entity_group': 'Disease',
  'score': 0.987484,
  'word': 'inflammatory bowel disease',
  'start': 866,
  'end': 892},
 {'entity_group': 'Disease',
  'score': 0.99387664,
  'word': 'crohns disease',
  'start': 904,
  'end': 918},
 {'entity_group': 'Disease',
  'score': 0.99451834,
  'word': 'ulcerative colitis',
  'start': 923,
  'end': 941},
 {'entity_group': 'Disease',
  'score': 0.99387145,
  'word': 'gastrointestinal cancer',
  'start': 957,
  'end': 980},
 {'entity_group': 'Disease',
  'score': 0.6751418,
  'word': '##yroptosis',
  'start': 1137,
  'end': 1146},
 {'entity_group': 'Disease',
  'score': 0.5804452,
  'word': '##plastic',
  'start': 1305,
  'end': 1312}]

In [ ]:
sentence = '''Abdominal aortic aneurysm, Acne
Acute cholecystitis
Acute lymphoblastic leukaemia
Acute lymphoblastic leukaemia: Children
Acute lymphoblastic leukaemia: Teenagers and young adults
Acute myeloid leukaemia
Acute myeloid leukaemia: Children
Acute myeloid leukaemia: Teenagers and young adults
Acute pancreatitis
Adenomyosis
Addison’s disease
Alcohol-related liver disease
Allergic rhinitis
Allergies
Alzheimer’s disease
Anal cancer
Anaphylaxis
Angioedema
Ankylosing spondylitis
Anorexia nervosa
Anxiety
Anxiety disorders in children
Appendicitis
Arthritis
Asbestosis
Asthma
Atopic eczema
Attention deficit hyperactivity disorder (ADHD)
Autistic spectrum disorder (ASD)'''
print("Sentence:",sentence)
token_classifier(sentence)

Sentence: Abdominal aortic aneurysm, Acne
Acute cholecystitis
Acute lymphoblastic leukaemia
Acute lymphoblastic leukaemia: Children
Acute lymphoblastic leukaemia: Teenagers and young adults
Acute myeloid leukaemia
Acute myeloid leukaemia: Children
Acute myeloid leukaemia: Teenagers and young adults
Acute pancreatitis
Adenomyosis
Addison’s disease
Alcohol-related liver disease
Allergic rhinitis
Allergies
Alzheimer’s disease
Anal cancer
Anaphylaxis
Angioedema
Ankylosing spondylitis
Anorexia nervosa
Anxiety
Anxiety disorders in children
Appendicitis
Arthritis
Asbestosis
Asthma
Atopic eczema
Attention deficit hyperactivity disorder (ADHD)
Autistic spectrum disorder (ASD)


[{'entity_group': 'Disease',
  'score': 0.986459,
  'word': 'abdominal aortic aneurysm',
  'start': 0,
  'end': 25},
 {'entity_group': 'Disease',
  'score': 0.9850693,
  'word': 'acne acute cholecystitis acute lymphoblastic leukaemia',
  'start': 27,
  'end': 81},
 {'entity_group': 'Disease',
  'score': 0.9431074,
  'word': 'acute lymphoblastic leukaemia',
  'start': 82,
  'end': 111},
 {'entity_group': 'Disease',
  'score': 0.9875766,
  'word': 'acute lymphoblastic leukaemia',
  'start': 122,
  'end': 151},
 {'entity_group': 'Disease',
  'score': 0.9603509,
  'word': 'acute myeloid leukaemia acute myeloid leukaemia',
  'start': 180,
  'end': 227},
 {'entity_group': 'Disease',
  'score': 0.9899391,
  'word': 'acute myeloid leukaemia',
  'start': 238,
  'end': 261},
 {'entity_group': 'Disease',
  'score': 0.98226786,
  'word': 'acute pancreatitis adenomyosis addison ’ s disease',
  'start': 290,
  'end': 338},
 {'entity_group': 'Disease',
  'score': 0.94890213,
  'word': 'alcohol - rela

In [ ]:
sentence = 'Diabetes is a chronic (long-lasting) health condition that affects how your body turns food into energy. Your body breaks down most of the food you eat into sugar (glucose) and releases it into your bloodstream. When your blood sugar goes up, it signals your pancreas to release insulin.'
print("Sentence:",sentence)
token_classifier(sentence)

Sentence: Diabetes is a chronic (long-lasting) health condition that affects how your body turns food into energy. Your body breaks down most of the food you eat into sugar (glucose) and releases it into your bloodstream. When your blood sugar goes up, it signals your pancreas to release insulin.


[{'entity_group': 'Disease',
  'score': 0.9107436,
  'word': 'diabetes',
  'start': 0,
  'end': 8},
 {'entity_group': 'Disease',
  'score': 0.56490815,
  'word': 'condition',
  'start': 44,
  'end': 53}]